# 予定
1. アクチュエータへの入力で一旦SB3で学習してみる
2. 逆運動学を利用して、エンドエフェクタの位置あるいは速度などで制御できるようにする
    - 実機でもしっかり取得できる情報じゃないと困る


In [ ]:
%matplotlib inline
import os
import mujoco
import numpy as np
from gymnasium.envs.mujoco import MujocoEnv
from gymnasium import spaces
from gymnasium.envs.registration import register
from gymnasium.spaces import Box
from typing import Dict, Union
import matplotlib.pyplot as plt
import cv2
from typing import Optional, Union

register(
    id='myenv-v0',
    entry_point='myenv.env:MyEnv'
)


DEFAULT_CAMERA_CONFIG = {
    "trackbodyid": -1,
    "distance": 4.0,
}

class MyRobotEnv(MujocoEnv):
    def __init__(
        self, 
        array=None,
        xml_path = "/workspace/ros2_ws/src/Air_Hockey_Challenge2025s/izumi/assets/main.xml",
        frame_skip: int = 5,
        default_camera_config: Dict[str, Union[float, int]] = DEFAULT_CAMERA_CONFIG,
        **kwargs,
    ):
        # 一度モデルを読み込んで観測次元を取得
        model = mujoco.MjModel.from_xml_path(xml_path)
        obs_dim = model.nq + model.nv
        observation_space = Box(low=-np.inf, high=np.inf, shape=(obs_dim,), dtype=np.float32)

        # 行動空間：joint velocities [-1, 1] 正規化
        # 現状はmodelのアクチュエータが認識していないので、model.nuがなくなっている
        # そのため, model.ctrlもないのでstepができない状態
        self.action_space = spaces.Box(
            low=-1.0, high=1.0, shape=model.actuator_actnum.shape, dtype=np.float32
        )
        self.array = array
        
        # 親クラスの初期化
        super().__init__(
            model_path=xml_path,
            frame_skip=frame_skip,
            observation_space=observation_space,
            render_mode="rgb_array",
            default_camera_config=default_camera_config,
            **kwargs,
        )
        
        self.metadata = {
            "render_modes": [
                "human",
                "rgb_array",
                "depth_array",
                "rgbd_tuple",
            ],
            "render_fps": int(np.round(1.0 / self.dt)),
        }
        

    def step(self, action):
        # 正規化されたactionをスケーリング
        #scaled_action = action * self.model.actuator_ctrlrange[:, 1]
        self.do_simulation(action, self.frame_skip)

        obs = self._get_obs()
        reward = self._compute_reward(obs, action)
        done = False
        info = {}

        return obs, reward, done, info

    def _get_obs(self):
        return np.concatenate([self.data.qpos.flat, self.data.qvel.flat])

    def _compute_reward(self, obs, action):
        # エンドエフェクタの位置を使用した報酬例
        site_id = mujoco.mj_name2id(self.model, mujoco.mjtObj.mjOBJ_SITE, "ee_site")
        end_effector_pos = self.data.site_xpos[site_id]
        goal = np.array([0.5, 0.0, 0.2])
        dist = np.linalg.norm(end_effector_pos - goal)
        return -dist

    def reset_model(self):
        # ランダム初期化
        qpos = self.array
        
        qvel = self.init_qvel
        self.set_state(qpos, qvel)
        return self._get_obs()

register(
    id='ArmHockey-v0',
    entry_point='myenv.env:MyEnv'
)

In [4]:
def render(images):
    for img in images:
        # 画像をuint8のBGRに変換（OpenCV用）
        img_bgr = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        
        cv2.imshow('Video', img_bgr)
        if cv2.waitKey(100) & 0xFF == ord('q'):
            break

    cv2.destroyAllWindows()

In [137]:
def print_joint(model):
    for i in range(model.njnt):
        name = mujoco.mj_id2name(model, mujoco.mjtObj.mjOBJ_JOINT, i)
        addr = model.jnt_qposadr[i]
        dof = model.jnt_dofadr[i]
        type_ = model.jnt_type[i]
        print(f"Joint {i}: name={name}, qpos address={addr}, dof address={dof}, type={type_}")

print_joint(env.model)

Joint 0: name=puck_x, qpos address=0, dof address=0, type=2
Joint 1: name=puck_y, qpos address=1, dof address=1, type=2
Joint 2: name=puck_yaw, qpos address=2, dof address=2, type=3
Joint 3: name=crane_x7_shoulder_fixed_part_pan_joint, qpos address=3, dof address=3, type=3
Joint 4: name=crane_x7_shoulder_revolute_part_tilt_joint, qpos address=4, dof address=4, type=3
Joint 5: name=crane_x7_upper_arm_revolute_part_twist_joint, qpos address=5, dof address=5, type=3
Joint 6: name=crane_x7_upper_arm_revolute_part_rotate_joint, qpos address=6, dof address=6, type=3
Joint 7: name=crane_x7_lower_arm_fixed_part_joint, qpos address=7, dof address=7, type=3
Joint 8: name=crane_x7_lower_arm_revolute_part_joint, qpos address=8, dof address=8, type=3
Joint 9: name=crane_x7_wrist_joint, qpos address=9, dof address=9, type=3
Joint 10: name=crane_x7_gripper_finger_a_joint, qpos address=10, dof address=10, type=3
Joint 11: name=crane_x7_gripper_finger_b_joint, qpos address=11, dof address=11, type=3


In [12]:
env = MyRobotEnv(
    #array=np.array([0,0,0,-np.pi,np.pi/3,0,np.pi/3,0,np.pi/6,0,0,0]),
    array=np.zeros(12),
)
print(env.init_qpos)
env.reset()
images = []
skip_frame = 5
model = env.model
data = env.data
ee_site_name = "ee_site"
ee_site_id = mujoco.mj_name2id(model, mujoco.mjtObj.mjOBJ_SITE, ee_site_name)
lambda_=0.1
Kp=100.0
Kd=3.0
target_pos=np.array([0.8,0,0.3])
dt = model.opt.timestep
jnt_range = model.jnt_range*0.95  # shape: (n_joints, 2)

mujoco.mj_resetData(model, data)

for i in range(1000):
    mujoco.mj_forward(model, data)

        # 現在のエンドエフェクタ位置を取得
    ee_pos = data.site_xpos[ee_site_id]
    error = target_pos - ee_pos
    #print(f"{np.linalg.norm(error)}")

    # if i == 0:
    #     print(f"⏳ 現在位置: {ee_pos}")
    #     print(f"ターゲットとの距離: {np.linalg.norm(error)}")

    lambda_fin = lambda_ * np.power(5, -np.linalg.norm(error))

    # サイト位置のJacobianを取得
    J_pos = np.zeros((3, model.nv))
    mujoco.mj_jacSite(model, data, J_pos, None, ee_site_id)

    # エンドエフェクタ速度
    ee_vel = J_pos @ data.qvel
    desired_acc = Kp * error - Kd * ee_vel

    # Damped Least Squares
    JJt = J_pos @ J_pos.T
    damped_inv = np.linalg.inv(JJt + lambda_fin**2 * np.eye(3))
    qvel_des = J_pos.T @ (damped_inv @ desired_acc)

    # ⚠️ ここで qvel_des を元に、次の位置 q + q̇ * dt を仮想計算
    q_next = data.qpos + qvel_des * dt

    # 関節の可動域にクリップ
    for j in range(model.nv):
        q_next[j] = np.clip(q_next[j], jnt_range[j, 0], jnt_range[j, 1])
    
    # クリップ後の q_next に対して、再び qvel_des を再計算（微分）
    qvel_des = (q_next - data.qpos) / dt

    # トルク制御に適用
    # トルク制限（アクチュエータ定義に従う）
    qvel = data.qvel[-model.nu:]
    torque = qvel_des[-model.nu:] - Kd * qvel
    print(f"トルク: {torque}")
    action = torque
    obs, reward, done, info = env.step(action)
    if i % skip_frame == 0:
        images.append(env.render())

render(images)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
トルク: [0.00000000e+00 3.01486283e+01 0.00000000e+00 1.65806350e-02
 0.00000000e+00 1.10370701e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00]
トルク: [ 2.35180270e-04  3.04193755e+01  2.37624414e-04  4.50175192e+00
  3.09355370e-04  7.76729333e-01  1.87418922e-04  6.45649538e-04
 -7.75498006e-04]
トルク: [-2.01457630e-03  3.32156134e+01 -2.04592367e-03  5.52043055e+00
 -3.00641146e-03  2.14761612e-01 -1.87561368e-03 -7.24412521e-03
  6.73922797e-03]
トルク: [ 2.09717853e-02  3.63448564e+01  2.10240560e-02 -2.92674642e+00
  1.45082945e-01 -3.95424485e-01  2.15404372e-02  8.12639842e-02
 -8.43267787e-02]
トルク: [ -2.02093264  43.95487893  -2.01826435 -18.29735559  -5.08969887
  -1.9769705   -2.56127405  -4.71632475   4.7119821 ]
トルク: [ 10.6260026   62.81335978   0.33270272 -55.58295884  -0.38638047
  -5.09903216   2.46941603   0.06774723  -0.07204127]
トルク: [  -1.29358041   91.1093278    -5.61568383 -110.9987457     5.88094599
   -9.52344965   -2.81202863   -4